# **NTDB TQP Parser**

### Importing required packages, defining file directories, and defining useful functions

In [ ]:
# Importing required packages
import os
import glob
import re
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
from collections import Counter
from tableone import TableOne

# Setting the file directory for this folder and for the raw data (stored in another HD because it's huge)
cwd = os.getcwd()
print(cwd)
raw_wd = "I:/BCM_projects/Bike-Injuries/NTDB"

In [ ]:
def csv_importer(filepath, rows):
    """
    Function to import a list of csv files as a named dictionary of dfs, viewing the shape and head of each df
    Inputs:
    filepath = full filepath to the CSV folder for each NTDB TQP year
    rows = number of rows to parse (either 10 or 'None' to parse all)\
    Outputs:
    print statements to display shape and head of dfs 
    named dictionary of all csv dfs in the csv folder
    """
    # Finding and printing a list of csv files
    csv_files = glob.glob(filepath + "/*.csv")
    print("csv file list:")
    [print(file) for file in csv_files][0]
    
    # Importing all csv files as a named dictionary of dfs, viewing the shape and head of each df
    df_dict = {}
    for csv in csv_files:
        df_name = re.findall("\\\\(.*?).csv", csv)[0]
        try:
            df = pd.read_csv(csv, nrows=rows)
            df.columns = [s.upper() for s in df.columns]
        except:
            df = pd.read_csv(csv, nrows=rows, encoding='latin-1')
            df.columns = [s.upper() for s in df.columns]
            print("\nAlternate encoding used for {}".format(df_name))
        df_dict[df_name] = df
        df_shape = str(df.shape)
        print("\n" + df_name + " - Dimensions: " + df_shape)
        display(df.head())
    return(df_dict)

In [ ]:
def stat_printer(df):
    """
    Function to display some statistics of dataframes.  Input df MUST have an "INC_KEY" column.
    Input: a dataframe with an INC_KEY column
    Outputs: print/display statements with statistics about the dataframe
    """
    x = len(df["INC_KEY"].unique())
    print("Shape: {0}\nUnique INC KEYS: {1}".format(str(df.shape), str(x)))
    display(df.head())

## **Importing and Cleaning Files for 2017**

In [ ]:
puf_2017 = csv_importer(filepath="I:/BCM_projects/Bike-Injuries/NTDB/PUF AY 2017/CSV", rows=None)

### Trauma

In [ ]:
# Selecting pedal cyclist injuries from ICD-10 codes
icd10_biker_ecodes = ["V10.0XXA", "V10.2XXA", "V10.4XXA", "V11.0XXA", "V11.1XXA", "V11.2XXA", "V11.4XXA", "V12.0XXA", "V12.4XXA", "V12.5XXA", 
                      "V12.9XXA", "V13.0XXA", "V13.1XXA", "V13.2XXA", "V13.3XXA", "V13.4XXA", "V13.5XXA", "V13.9XXA", "V14.0XXA", "V14.4XXA", 
                      "V14.5XXA", "V14.9XXA", "V15.4XXA", "V16.0XXA", "V16.4XXA", "V17.0XXA", "V17.1XXA", "V17.2XXA", "V17.3XXA", "V17.4XXA", 
                      "V17.9XXA", "V18.0XXA", "V18.1XXA", "V18.2XXA", "V18.3XXA", "V18.4XXA", "V18.5XXA", "V18.9XXA", "V19.00XA", "V19.09XA", 
                      "V19.3XXA", "V19.40XA", "V19.49XA", "V19.59XA", "V19.60XA", "V19.88XA", "V19.9XXA"]
icd10_ecode_bikes = puf_2017["PUF_ECODE_LOOKUP"]
icd10_ecode_bikes = icd10_ecode_bikes[icd10_ecode_bikes["ECODE"].isin(icd10_biker_ecodes)]
ecode_dict = dict(list(zip(icd10_ecode_bikes["ECODE"], icd10_ecode_bikes["ECODE_DESC"])))
#icd10_ecode_bikes

# Filtering the major trauma df for bike injuries and annotating with ecode descriptions
trauma = puf_2017["PUF_TRAUMA"]
trauma_bikes = trauma[trauma["PRIMARYECODEICD10"].isin(icd10_biker_ecodes)]
trauma_bikes["ECODE_DESC"] = trauma_bikes["PRIMARYECODEICD10"].map(ecode_dict)
trauma_bikes["Year"] = 2017
inc_keys = pd.Series(trauma_bikes["INC_KEY"]) #getting a list of INC_KEYS to filter other dfs
year = 2017
n_total = len(trauma["INC_KEY"].unique())
n_bikers = len(trauma_bikes["INC_KEY"].unique())
frac_bikers = np.round((n_bikers / n_total) * 100, 3)
print("In the year {0} there were {1}/{2} pedalcylist trauma injuries / total trauama injuries ({3}%)".format(year, n_bikers, n_total, frac_bikers))
stat_printer(trauma_bikes)

# Optional export to CSV
#trauma_bikes.to_csv(cwd+"/TQP_Files/main/2017_trauma.csv", index=False)

### Injury Severity Score (AIS)

In [ ]:
# Filtering PUF_AISDIAGNOSIS for biker INC_KEYS
ais = puf_2017["PUF_AISDIAGNOSIS"]
ais_bikes = ais[ais["INC_KEY"].isin(inc_keys)]

# Merging with AISPREDOT descriptions
ais_lookup = puf_2017["PUF_AISDIAGNOSIS_LOOKUP"]
ais_lookup["AISPREDOT"] = ais_lookup["AISPREDOT"].astype(float)
ais_lookup = ais_lookup.drop(["AISSEVERITY", "AISVERSION"], axis=1)
ais_bikes = pd.merge(ais_bikes, ais_lookup, how="left", on="AISPREDOT")

# Merging with AIS05TO98_CROSSWALK
ais_cross = puf_2017["PUF_AIS05TO98_CROSSWALK"]
ais_cross.columns = ['AISPREDOT', 'AIS05_SEVERITY', 'AIS98_PREDOT', 'AIS98_SEVERITY', 'AIS98_DESCRIPTION']
ais_cross["AISPREDOT"] = ais_cross["AISPREDOT"].astype(float)
ais_cross = ais_cross.drop("AIS05_SEVERITY", axis=1)
ais_bikes = pd.merge(ais_bikes, ais_cross, how="left", on="AISPREDOT")
ais_bikes["Year"] = 2017
stat_printer(ais_bikes)

# Optional export to CSV
#ais_bikes.to_csv(cwd+"/TQP_Files/ais/2017_AIS.csv", index=False)

### Diagnosis

In [ ]:
# Filtering PUF_AISDIAGNOSIS for biker INC_KEYS
diag = puf_2017["PUF_ICDDIAGNOSIS"]
diag_bikes = diag[diag["INC_KEY"].isin(inc_keys)]

# Merging with ICDDIAGNOSISCODE descriptions
diag_loopup = puf_2017["PUF_ICDDIAGNOSIS_LOOKUP"]
diag_loopup = diag_loopup.drop("ICD_VERSION", axis=1)
diag_bikes = pd.merge(diag_bikes, diag_loopup, how="left", on="ICDDIAGNOSISCODE")
diag_bikes["Year"] = 2017
stat_printer(diag_bikes)

# Optional export to CSV
#diag_bikes.to_csv(cwd+"/TQP_Files/diagnosis/2017_diagnosis.csv", index=False)

### Procedures

In [ ]:
# Filtering PUF_ICDPROCEDURE for biker INC_KEYS
proced = puf_2017["PUF_ICDPROCEDURE"]
proced_bikes = proced[proced["INC_KEY"].isin(inc_keys)]

# Merging with ICDPROCEDURECODE descriptions
proced_loopup = puf_2017["PUF_ICDPROCEDURE_LOOKUP"]
proced_loopup = proced_loopup.drop("ICD_VERSION", axis=1)
proced_bikes = pd.merge(proced_bikes, proced_loopup, how="left", on="ICDPROCEDURECODE")
proced_bikes["Year"] = 2017
stat_printer(proced_bikes)

# Optional export to CSV
#proced_bikes.to_csv(cwd+"/TQP_Files/procedures/2017_procedures.csv", index=False)

## **Importing and Cleaning Files for 2018**

In [ ]:
puf_2018 = csv_importer(filepath="I:/BCM_projects/Bike-Injuries/NTDB/PUF AY 2018/CSV", rows=None)

### Trauma

In [ ]:
# Selecting pedal cyclist injuries from ICD-10 codes
icd10_biker_ecodes = ["V10.0XXA", "V10.2XXA", "V10.4XXA", "V11.0XXA", "V11.1XXA", "V11.2XXA", "V11.4XXA", "V12.0XXA", "V12.4XXA", "V12.5XXA", 
                      "V12.9XXA", "V13.0XXA", "V13.1XXA", "V13.2XXA", "V13.3XXA", "V13.4XXA", "V13.5XXA", "V13.9XXA", "V14.0XXA", "V14.4XXA", 
                      "V14.5XXA", "V14.9XXA", "V15.4XXA", "V16.0XXA", "V16.4XXA", "V17.0XXA", "V17.1XXA", "V17.2XXA", "V17.3XXA", "V17.4XXA", 
                      "V17.9XXA", "V18.0XXA", "V18.1XXA", "V18.2XXA", "V18.3XXA", "V18.4XXA", "V18.5XXA", "V18.9XXA", "V19.00XA", "V19.09XA", 
                      "V19.3XXA", "V19.40XA", "V19.49XA", "V19.59XA", "V19.60XA", "V19.88XA", "V19.9XXA"]
icd10_ecode_bikes = puf_2018["PUF_ECODE_LOOKUP"]
icd10_ecode_bikes = icd10_ecode_bikes[icd10_ecode_bikes["ECODE"].isin(icd10_biker_ecodes)]
ecode_dict = dict(list(zip(icd10_ecode_bikes["ECODE"], icd10_ecode_bikes["ECODE_DESC"])))

# Filtering the major trauma df for bike injuries and annotating with ecode descriptions
trauma = puf_2018["PUF_TRAUMA"]
trauma_bikes = trauma[trauma["PRIMARYECODEICD10"].isin(icd10_biker_ecodes)]
trauma_bikes["ECODE_DESC"] = trauma_bikes["PRIMARYECODEICD10"].map(ecode_dict)
trauma_bikes["Year"] = 2018
inc_keys = pd.Series(trauma_bikes["INC_KEY"]) #getting a list of INC_KEYS to filter other dfs
year = 2018
n_total = len(trauma["INC_KEY"].unique())
n_bikers = len(trauma_bikes["INC_KEY"].unique())
frac_bikers = np.round((n_bikers / n_total) * 100, 3)
print("In the year {0} there were {1}/{2} pedalcylist trauma injuries / total trauama injuries ({3}%)".format(year, n_bikers, n_total, frac_bikers))
stat_printer(trauma_bikes)

# Optional export to CSV
#trauma_bikes.to_csv(cwd+"/TQP_Files/main/2018_trauma.csv", index=False)

### Injury Severity Score (AIS)

In [ ]:
# Filtering PUF_AISDIAGNOSIS for biker INC_KEYS
ais = puf_2018["PUF_AISDIAGNOSIS"]
ais_bikes = ais[ais["INC_KEY"].isin(inc_keys)]

# Merging with AISPREDOT descriptions
ais_lookup = puf_2018["PUF_AISDIAGNOSIS_LOOKUP"]
ais_lookup["AISPREDOT"] = ais_lookup["AISPREDOT"].astype(float)
ais_lookup = ais_lookup.drop(["AISSEVERITY", "AISVERSION"], axis=1)
ais_bikes = pd.merge(ais_bikes, ais_lookup, how="left", on="AISPREDOT")

# Merging with AIS05TO98_CROSSWALK
ais_cross = puf_2018["PUF_AIS05TO98_CROSSWALK"]
ais_cross.columns = ['AISPREDOT', 'AIS05_SEVERITY', 'AIS98_PREDOT', 'AIS98_SEVERITY', 'AIS98_DESCRIPTION']
ais_cross["AISPREDOT"] = ais_cross["AISPREDOT"].astype(float)
ais_cross = ais_cross.drop("AIS05_SEVERITY", axis=1)
ais_bikes = pd.merge(ais_bikes, ais_cross, how="left", on="AISPREDOT")
ais_bikes["Year"] = 2018
stat_printer(ais_bikes)

# Optional export to CSV
#ais_bikes.to_csv(cwd+"/TQP_Files/ais/2018_AIS.csv", index=False)

### Diagnosis

In [ ]:
# Filtering PUF_AISDIAGNOSIS for biker INC_KEYS
diag = puf_2018["PUF_ICDDIAGNOSIS"]
diag_bikes = diag[diag["INC_KEY"].isin(inc_keys)]

# Merging with ICDDIAGNOSISCODE descriptions
diag_loopup = puf_2018["PUF_ICDDIAGNOSIS_LOOKUP"]
diag_loopup = diag_loopup.drop("ICD_VERSION", axis=1)
diag_bikes = pd.merge(diag_bikes, diag_loopup, how="left", on="ICDDIAGNOSISCODE")
diag_bikes["Year"] = 2018
stat_printer(diag_bikes)

# Optional export to CSV
#diag_bikes.to_csv(cwd+"/TQP_Files/diagnosis/2018_diagnosis.csv", index=False)

### Procedures

In [ ]:
# Filtering PUF_ICDPROCEDURE for biker INC_KEYS
proced = puf_2018["PUF_ICDPROCEDURE"]
proced_bikes = proced[proced["INC_KEY"].isin(inc_keys)]

# Merging with ICDPROCEDURECODE descriptions
proced_loopup = puf_2018["PUF_ICDPROCEDURE_LOOKUP"]
proced_loopup = proced_loopup.drop("ICD_VERSION", axis=1)
proced_bikes = pd.merge(proced_bikes, proced_loopup, how="left", on="ICDPROCEDURECODE")
proced_bikes["Year"] = 2018
stat_printer(proced_bikes)

# Optional export to CSV
#proced_bikes.to_csv(cwd+"/TQP_Files/procedures/2018_procedures.csv", index=False)

## **Importing and Cleaning Files for 2019**

In [ ]:
puf_2019 = csv_importer(filepath="I:/BCM_projects/Bike-Injuries/NTDB/PUF AY 2019/CSV", rows=None)

### Trauma

In [ ]:
# Selecting pedal cyclist injuries from ICD-10 codes
icd10_biker_ecodes = ["V10.0XXA", "V10.2XXA", "V10.4XXA", "V11.0XXA", "V11.1XXA", "V11.2XXA", "V11.4XXA", "V12.0XXA", "V12.4XXA", "V12.5XXA", 
                      "V12.9XXA", "V13.0XXA", "V13.1XXA", "V13.2XXA", "V13.3XXA", "V13.4XXA", "V13.5XXA", "V13.9XXA", "V14.0XXA", "V14.4XXA", 
                      "V14.5XXA", "V14.9XXA", "V15.4XXA", "V16.0XXA", "V16.4XXA", "V17.0XXA", "V17.1XXA", "V17.2XXA", "V17.3XXA", "V17.4XXA", 
                      "V17.9XXA", "V18.0XXA", "V18.1XXA", "V18.2XXA", "V18.3XXA", "V18.4XXA", "V18.5XXA", "V18.9XXA", "V19.00XA", "V19.09XA", 
                      "V19.3XXA", "V19.40XA", "V19.49XA", "V19.59XA", "V19.60XA", "V19.88XA", "V19.9XXA"]
icd10_ecode_bikes = puf_2019["PUF_ECODE_LOOKUP"]
icd10_ecode_bikes = icd10_ecode_bikes[icd10_ecode_bikes["ECODE"].isin(icd10_biker_ecodes)]
ecode_dict = dict(list(zip(icd10_ecode_bikes["ECODE"], icd10_ecode_bikes["ECODE_DESC"])))

# Filtering the major trauma df for bike injuries and annotating with ecode descriptions
trauma = puf_2019["PUF_TRAUMA"]
trauma_bikes = trauma[trauma["PRIMARYECODEICD10"].isin(icd10_biker_ecodes)]
trauma_bikes["ECODE_DESC"] = trauma_bikes["PRIMARYECODEICD10"].map(ecode_dict)
trauma_bikes["Year"] = 2019
inc_keys = pd.Series(trauma_bikes["INC_KEY"]) #getting a list of INC_KEYS to filter other dfs
year = 2019
n_total = len(trauma["INC_KEY"].unique())
n_bikers = len(trauma_bikes["INC_KEY"].unique())
frac_bikers = np.round((n_bikers / n_total) * 100, 3)
print("In the year {0} there were {1}/{2} pedalcylist trauma injuries / total trauama injuries ({3}%)".format(year, n_bikers, n_total, frac_bikers))
stat_printer(trauma_bikes)

# Optional export to CSV
#trauma_bikes.to_csv(cwd+"/TQP_Files/main/2019_trauma.csv", index=False)

### Injury Severity Score (AIS)

In [ ]:
# Filtering PUF_AISDIAGNOSIS for biker INC_KEYS
ais = puf_2019["PUF_AISDIAGNOSIS"]
ais_bikes = ais[ais["INC_KEY"].isin(inc_keys)]

# Merging with AISPREDOT descriptions
ais_lookup = puf_2019["PUF_AISDIAGNOSIS_LOOKUP"]
ais_lookup["AISPREDOT"] = ais_lookup["AISPREDOT"].astype(float)
ais_lookup = ais_lookup.drop(["AISSEVERITY", "AISVERSION"], axis=1)
ais_bikes = pd.merge(ais_bikes, ais_lookup, how="left", on="AISPREDOT")

# Merging with AIS05TO98_CROSSWALK
ais_cross = puf_2019["PUF_AIS05TO98_CROSSWALK"]
ais_cross.columns = ['AISPREDOT', 'AIS05_SEVERITY', 'AIS98_PREDOT', 'AIS98_SEVERITY', 'AIS98_DESCRIPTION']
ais_cross["AISPREDOT"] = ais_cross["AISPREDOT"].astype(float)
ais_cross = ais_cross.drop("AIS05_SEVERITY", axis=1)
ais_bikes = pd.merge(ais_bikes, ais_cross, how="left", on="AISPREDOT")
ais_bikes["Year"] = 2019
stat_printer(ais_bikes)

# Optional export to CSV
#ais_bikes.to_csv(cwd+"/TQP_Files/ais/2019_AIS.csv", index=False)

### Hospital events

In [ ]:
# Filtering PUF_HOSPITALEVENTS for biker INC_KEYS
events = puf_2019["PUF_HOSPITALEVENTS"]
events_bikes = events[events["INC_KEY"].isin(inc_keys)]
events_bikes["Year"] = 2019
stat_printer(events_bikes)

# Optional export to CSV
#events_bikes.to_csv(cwd+"/TQP_Files/events/2019_events.csv", index=False)

### Diagnosis

In [ ]:
# Filtering PUF_AISDIAGNOSIS for biker INC_KEYS
diag = puf_2019["PUF_ICDDIAGNOSIS"]
diag_bikes = diag[diag["INC_KEY"].isin(inc_keys)]

# Merging with ICDDIAGNOSISCODE descriptions
diag_loopup = puf_2019["PUF_ICDDIAGNOSIS_LOOKUP"]
diag_loopup = diag_loopup.drop("ICD_VERSION", axis=1)
diag_bikes = pd.merge(diag_bikes, diag_loopup, how="left", on="ICDDIAGNOSISCODE")
diag_bikes["Year"] = 2019
stat_printer(diag_bikes)

# Optional export to CSV
#diag_bikes.to_csv(cwd+"/TQP_Files/diagnosis/2019_diagnosis.csv", index=False)

### Procedures

In [ ]:
# Filtering PUF_ICDPROCEDURE for biker INC_KEYS
proced = puf_2019["PUF_ICDPROCEDURE"]
proced_bikes = proced[proced["INC_KEY"].isin(inc_keys)]

# Merging with ICDPROCEDURECODE descriptions
proced_loopup = puf_2019["PUF_ICDPROCEDURE_LOOKUP"]
proced_loopup = proced_loopup.drop("ICD_VERSION", axis=1)
proced_bikes = pd.merge(proced_bikes, proced_loopup, how="left", on="ICDPROCEDURECODE")
proced_bikes["Year"] = 2019
stat_printer(proced_bikes)

# Optional export to CSV
#proced_bikes.to_csv(cwd+"/TQP_Files/procedures/2019_procedures.csv", index=False)

### Preexisting Conditions

In [ ]:
# Filtering PUF_HOSPITALEVENTS for biker INC_KEYS
pre = puf_2019["PUF_PREEXISTINGCONDITIONS"]
pre_bikes = pre[pre["INC_KEY"].isin(inc_keys)]
pre_bikes["Year"] = 2019
stat_printer(pre_bikes)

# Optional export to CSV
#pre_bikes.to_csv(cwd+"/TQP_Files/preexisting/2019_preexisting_conditions.csv", index=False)

## **Importing and Cleaning Files for 2020**

In [ ]:
puf_2020 = csv_importer(filepath="I:/BCM_projects/Bike-Injuries/NTDB/PUF AY 2020/CSV", rows=None)

### Trauma

In [ ]:
# Selecting pedal cyclist injuries from ICD-10 codes
icd10_biker_ecodes = ["V10.0XXA", "V10.2XXA", "V10.4XXA", "V11.0XXA", "V11.1XXA", "V11.2XXA", "V11.4XXA", "V12.0XXA", "V12.4XXA", "V12.5XXA", 
                      "V12.9XXA", "V13.0XXA", "V13.1XXA", "V13.2XXA", "V13.3XXA", "V13.4XXA", "V13.5XXA", "V13.9XXA", "V14.0XXA", "V14.4XXA", 
                      "V14.5XXA", "V14.9XXA", "V15.4XXA", "V16.0XXA", "V16.4XXA", "V17.0XXA", "V17.1XXA", "V17.2XXA", "V17.3XXA", "V17.4XXA", 
                      "V17.9XXA", "V18.0XXA", "V18.1XXA", "V18.2XXA", "V18.3XXA", "V18.4XXA", "V18.5XXA", "V18.9XXA", "V19.00XA", "V19.09XA", 
                      "V19.3XXA", "V19.40XA", "V19.49XA", "V19.59XA", "V19.60XA", "V19.88XA", "V19.9XXA"]
icd10_ecode_bikes = puf_2020["PUF_ECODE_LOOKUP"]
icd10_ecode_bikes = icd10_ecode_bikes[icd10_ecode_bikes["ECODE"].isin(icd10_biker_ecodes)]
ecode_dict = dict(list(zip(icd10_ecode_bikes["ECODE"], icd10_ecode_bikes["ECODE_DESC"])))

# Filtering the major trauma df for bike injuries and annotating with ecode descriptions
trauma = puf_2020["PUF_TRAUMA"]
trauma_bikes = trauma[trauma["PRIMARYECODEICD10"].isin(icd10_biker_ecodes)]
trauma_bikes["ECODE_DESC"] = trauma_bikes["PRIMARYECODEICD10"].map(ecode_dict)
trauma_bikes["Year"] = 2020
inc_keys = pd.Series(trauma_bikes["INC_KEY"]) #getting a list of INC_KEYS to filter other dfs
year = 2020
n_total = len(trauma["INC_KEY"].unique())
n_bikers = len(trauma_bikes["INC_KEY"].unique())
frac_bikers = np.round((n_bikers / n_total) * 100, 3)
print("In the year {0} there were {1}/{2} pedalcylist trauma injuries / total trauama injuries ({3}%)".format(year, n_bikers, n_total, frac_bikers))
stat_printer(trauma_bikes)

# Optional export to CSV
#trauma_bikes.to_csv(cwd+"/TQP_Files/main/2020_trauma.csv", index=False)

### Injury Severity Score (AIS)

In [ ]:
# Filtering PUF_AISDIAGNOSIS for biker INC_KEYS
ais = puf_2020["PUF_AISDIAGNOSIS"]
ais_bikes = ais[ais["INC_KEY"].isin(inc_keys)]
ais_bikes["AISPREDOT"] = ais_bikes["AISPREDOT"].astype(float)

# Merging with AISPREDOT descriptions
ais_lookup = puf_2020["PUF_AISDIAGNOSIS_LOOKUP"]
ais_lookup["AISPREDOT"] = ais_lookup["AISPREDOT"].astype(float)
ais_lookup = ais_lookup.drop(["AISSEVERITY", "AISVERSION"], axis=1)
ais_bikes = pd.merge(ais_bikes, ais_lookup, how="left", on="AISPREDOT")

# Merging with AIS05TO98_CROSSWALK
ais_cross = puf_2020["PUF_AIS15TO05_CROSSWALK"]
ais_cross.columns = ['AIS15_PREDOT', 'AIS15_SEVERITY', 'AISPREDOT', 'AIS05_SEVERITY', 'AIS15_DESCRIPTION']
ais_cross["AISPREDOT"] = ais_cross["AISPREDOT"].replace("None", np.nan)
ais_cross["AISPREDOT"] = ais_cross["AISPREDOT"].astype(float)
ais_cross = ais_cross.drop("AIS05_SEVERITY", axis=1)
ais_bikes = pd.merge(ais_bikes, ais_cross, how="left", on="AISPREDOT")
ais_bikes["Year"] = 2020
stat_printer(ais_bikes)

# Optional export to CSV
#ais_bikes.to_csv(cwd+"/TQP_Files/ais/2020_AIS.csv", index=False)

### Hospital events

In [ ]:
# Filtering PUF_HOSPITALEVENTS for biker INC_KEYS
events = puf_2020["PUF_HOSPITALEVENTS"]
events_bikes = events[events["INC_KEY"].isin(inc_keys)]
events_bikes["Year"] = 2020
stat_printer(events_bikes)

# Optional export to CSV
#events_bikes.to_csv(cwd+"/TQP_Files/events/2020_events.csv", index=False)

### Diagnosis

In [ ]:
# Filtering PUF_AISDIAGNOSIS for biker INC_KEYS
diag = puf_2020["PUF_ICDDIAGNOSIS"]
diag_bikes = diag[diag["INC_KEY"].isin(inc_keys)]

# Merging with ICDDIAGNOSISCODE descriptions
diag_loopup = puf_2020["PUF_ICDDIAGNOSIS_LOOKUP"]
diag_loopup = diag_loopup.drop("ICD_VERSION", axis=1)
diag_bikes = pd.merge(diag_bikes, diag_loopup, how="left", on="ICDDIAGNOSISCODE")
diag_bikes["Year"] = 2020
stat_printer(diag_bikes)

# Optional export to CSV
#diag_bikes.to_csv(cwd+"/TQP_Files/diagnosis/2020_diagnosis.csv", index=False)

### Procedures

In [ ]:
# Filtering PUF_ICDPROCEDURE for biker INC_KEYS
proced = puf_2020["PUF_ICDPROCEDURE"]
proced_bikes = proced[proced["INC_KEY"].isin(inc_keys)]

# Merging with ICDPROCEDURECODE descriptions
proced_loopup = puf_2020["PUF_ICDPROCEDURE_LOOKUP"]
proced_loopup = proced_loopup.drop("ICD_VERSION", axis=1)
proced_bikes = pd.merge(proced_bikes, proced_loopup, how="left", on="ICDPROCEDURECODE")
proced_bikes["Year"] = 2020
stat_printer(proced_bikes)

# Optional export to CSV
#proced_bikes.to_csv(cwd+"/TQP_Files/procedures/2020_procedures.csv", index=False)

### Preexisting Conditions

In [ ]:
# Filtering PUF_HOSPITALEVENTS for biker INC_KEYS
pre = puf_2020["PUF_PREEXISTINGCONDITIONS"]
pre_bikes = pre[pre["INC_KEY"].isin(inc_keys)]
pre_bikes["Year"] = 2020
stat_printer(pre_bikes)

# Optional export to CSV
#pre_bikes.to_csv(cwd+"/TQP_Files/preexisting/2020_preexisting_conditions.csv", index=False)

## **Importing and Cleaning Files for 2021**

In [ ]:
puf_2021 = csv_importer(filepath="I:/BCM_projects/Bike-Injuries/NTDB/PUF AY 2021/CSV", rows=None)

### Trauma

In [ ]:
# Selecting pedal cyclist injuries from ICD-10 codes
icd10_biker_ecodes = ["V10.0XXA", "V10.2XXA", "V10.4XXA", "V11.0XXA", "V11.1XXA", "V11.2XXA", "V11.4XXA", "V12.0XXA", "V12.4XXA", "V12.5XXA", 
                      "V12.9XXA", "V13.0XXA", "V13.1XXA", "V13.2XXA", "V13.3XXA", "V13.4XXA", "V13.5XXA", "V13.9XXA", "V14.0XXA", "V14.4XXA", 
                      "V14.5XXA", "V14.9XXA", "V15.4XXA", "V16.0XXA", "V16.4XXA", "V17.0XXA", "V17.1XXA", "V17.2XXA", "V17.3XXA", "V17.4XXA", 
                      "V17.9XXA", "V18.0XXA", "V18.1XXA", "V18.2XXA", "V18.3XXA", "V18.4XXA", "V18.5XXA", "V18.9XXA", "V19.00XA", "V19.09XA", 
                      "V19.3XXA", "V19.40XA", "V19.49XA", "V19.59XA", "V19.60XA", "V19.88XA", "V19.9XXA"]
icd10_ecode_bikes = puf_2021["PUF_ECODE_LOOKUP"]
icd10_ecode_bikes = icd10_ecode_bikes[icd10_ecode_bikes["ECODE"].isin(icd10_biker_ecodes)]
ecode_dict = dict(list(zip(icd10_ecode_bikes["ECODE"], icd10_ecode_bikes["ECODE_DESC"])))

# Filtering the major trauma df for bike injuries and annotating with ecode descriptions
trauma = puf_2021["PUF_TRAUMA"]
trauma_bikes = trauma[trauma["PRIMARYECODEICD10"].isin(icd10_biker_ecodes)]
trauma_bikes["ECODE_DESC"] = trauma_bikes["PRIMARYECODEICD10"].map(ecode_dict)
trauma_bikes["Year"] = 2021
inc_keys = pd.Series(trauma_bikes["INC_KEY"]) #getting a list of INC_KEYS to filter other dfs
year = 2021
n_total = len(trauma["INC_KEY"].unique())
n_bikers = len(trauma_bikes["INC_KEY"].unique())
frac_bikers = np.round((n_bikers / n_total) * 100, 3)
print("In the year {0} there were {1}/{2} pedalcylist trauma injuries / total trauama injuries ({3}%)".format(year, n_bikers, n_total, frac_bikers))
stat_printer(trauma_bikes)

# Optional export to CSV
#trauma_bikes.to_csv(cwd+"/TQP_Files/main/2021_trauma.csv", index=False)

### Injury Severity Score (AIS)

In [ ]:
# Filtering PUF_AISDIAGNOSIS for biker INC_KEYS
ais = puf_2021["PUF_AISDIAGNOSIS"]
ais_bikes = ais[ais["INC_KEY"].isin(inc_keys)]
ais_bikes["AISPREDOT"] = ais_bikes["AISPREDOT"].astype(float)

# Merging with AISPREDOT descriptions
ais_lookup = puf_2021["PUF_AISDIAGNOSIS_LOOKUP"]
ais_lookup["AISPREDOT"] = ais_lookup["AISPREDOT"].astype(float)
ais_lookup = ais_lookup.drop(["AISSEVERITY", "AISVERSION"], axis=1)
ais_bikes = pd.merge(ais_bikes, ais_lookup, how="left", on="AISPREDOT")

# Merging with AIS05TO98_CROSSWALK
ais_cross = puf_2021["PUF_AIS15TO05_CROSSWALK"]
ais_cross.columns = ['AIS15_PREDOT', 'AIS15_SEVERITY', 'AISPREDOT', 'AIS05_SEVERITY', 'AIS15_DESCRIPTION']
ais_cross["AISPREDOT"] = ais_cross["AISPREDOT"].replace("None", np.nan)
ais_cross["AISPREDOT"] = ais_cross["AISPREDOT"].astype(float)
ais_cross = ais_cross.drop("AIS05_SEVERITY", axis=1)
ais_bikes = pd.merge(ais_bikes, ais_cross, how="left", on="AISPREDOT")
ais_bikes["Year"] = 2021
stat_printer(ais_bikes)

# Optional export to CSV
#ais_bikes.to_csv(cwd+"/TQP_Files/ais/2021_AIS.csv", index=False)

### Hospital events

In [ ]:
# Filtering PUF_HOSPITALEVENTS for biker INC_KEYS
events = puf_2021["PUF_HOSPITALEVENTS"]
events_bikes = events[events["INC_KEY"].isin(inc_keys)]
events_bikes["Year"] = 2021
stat_printer(events_bikes)

# Optional export to CSV
#events_bikes.to_csv(cwd+"/TQP_Files/events/2021_events.csv", index=False)

### Diagnosis

In [ ]:
# Filtering PUF_AISDIAGNOSIS for biker INC_KEYS
diag = puf_2021["PUF_ICDDIAGNOSIS"]
diag_bikes = diag[diag["INC_KEY"].isin(inc_keys)]

# Merging with ICDDIAGNOSISCODE descriptions
diag_loopup = puf_2021["PUF_ICDDIAGNOSIS_LOOKUP"]
diag_loopup = diag_loopup.drop("ICD_VERSION", axis=1)
diag_bikes = pd.merge(diag_bikes, diag_loopup, how="left", on="ICDDIAGNOSISCODE")
diag_bikes["Year"] = 2021
stat_printer(diag_bikes)

# Optional export to CSV
#diag_bikes.to_csv(cwd+"/TQP_Files/diagnosis/2021_diagnosis.csv", index=False)

### Procedures

In [ ]:
# Filtering PUF_ICDPROCEDURE for biker INC_KEYS
proced = puf_2021["PUF_ICDPROCEDURE"]
proced_bikes = proced[proced["INC_KEY"].isin(inc_keys)]

# Merging with ICDPROCEDURECODE descriptions
proced_loopup = puf_2021["PUF_ICDPROCEDURE_LOOKUP"]
proced_loopup = proced_loopup.drop("ICD_VERSION", axis=1)
proced_bikes = pd.merge(proced_bikes, proced_loopup, how="left", on="ICDPROCEDURECODE")
proced_bikes["Year"] = 2021
stat_printer(proced_bikes)

# Optional export to CSV
#proced_bikes.to_csv(cwd+"/TQP_Files/procedures/2021_procedures.csv", index=False)

### Preexisting Conditions

In [ ]:
# Filtering PUF_HOSPITALEVENTS for biker INC_KEYS
pre = puf_2021["PUF_PREEXISTINGCONDITIONS"]
pre_bikes = pre[pre["INC_KEY"].isin(inc_keys)]
pre_bikes["Year"] = 2021
stat_printer(pre_bikes)

# Optional export to CSV
#pre_bikes.to_csv(cwd+"/TQP_Files/preexisting/2021_preexisting_conditions.csv", index=False)

## **Importing and Cleaning Files for 2022**

In [ ]:
puf_2022 = csv_importer(filepath="I:/BCM_projects/Bike-Injuries/NTDB/PUF AY 2022/CSV", rows=None)

### Trauma

In [ ]:
# Selecting pedal cyclist injuries from ICD-10 codes
icd10_biker_ecodes = ["V10.0XXA", "V10.2XXA", "V10.4XXA", "V11.0XXA", "V11.1XXA", "V11.2XXA", "V11.4XXA", "V12.0XXA", "V12.4XXA", "V12.5XXA", 
                      "V12.9XXA", "V13.0XXA", "V13.1XXA", "V13.2XXA", "V13.3XXA", "V13.4XXA", "V13.5XXA", "V13.9XXA", "V14.0XXA", "V14.4XXA", 
                      "V14.5XXA", "V14.9XXA", "V15.4XXA", "V16.0XXA", "V16.4XXA", "V17.0XXA", "V17.1XXA", "V17.2XXA", "V17.3XXA", "V17.4XXA", 
                      "V17.9XXA", "V18.0XXA", "V18.1XXA", "V18.2XXA", "V18.3XXA", "V18.4XXA", "V18.5XXA", "V18.9XXA", "V19.00XA", "V19.09XA", 
                      "V19.3XXA", "V19.40XA", "V19.49XA", "V19.59XA", "V19.60XA", "V19.88XA", "V19.9XXA"]
icd10_ecode_bikes = puf_2022["PUF_ECODE_LOOKUP"]
icd10_ecode_bikes = icd10_ecode_bikes[icd10_ecode_bikes["ECODE"].isin(icd10_biker_ecodes)]
ecode_dict = dict(list(zip(icd10_ecode_bikes["ECODE"], icd10_ecode_bikes["ECODE_DESC"])))

# Filtering the major trauma df for bike injuries and annotating with ecode descriptions
trauma = puf_2022["PUF_TRAUMA"]
trauma_bikes = trauma[trauma["PRIMARYECODEICD10"].isin(icd10_biker_ecodes)]
trauma_bikes["ECODE_DESC"] = trauma_bikes["PRIMARYECODEICD10"].map(ecode_dict)
trauma_bikes["Year"] = 2022
inc_keys = pd.Series(trauma_bikes["INC_KEY"]) #getting a list of INC_KEYS to filter other dfs
year = 2022
n_total = len(trauma["INC_KEY"].unique())
n_bikers = len(trauma_bikes["INC_KEY"].unique())
frac_bikers = np.round((n_bikers / n_total) * 100, 3)
print("In the year {0} there were {1}/{2} pedalcylist trauma injuries / total trauama injuries ({3}%)".format(year, n_bikers, n_total, frac_bikers))
stat_printer(trauma_bikes)

# Optional export to CSV
#trauma_bikes.to_csv(cwd+"/TQP_Files/main/2022_trauma.csv", index=False)

### Injury Severity Score (AIS)

In [ ]:
# Filtering PUF_AISDIAGNOSIS for biker INC_KEYS
ais = puf_2022["PUF_AISDIAGNOSIS"]
ais_bikes = ais[ais["INC_KEY"].isin(inc_keys)]
ais_bikes["AISPREDOT"] = ais_bikes["AISPREDOT"].astype(float)

# Merging with AISPREDOT descriptions
ais_lookup = puf_2022["PUF_AISDIAGNOSIS_LOOKUP"]
ais_lookup["AISPREDOT"] = ais_lookup["AISPREDOT"].astype(float)
ais_lookup = ais_lookup.drop(["AISSEVERITY", "AISVERSION"], axis=1)
ais_bikes = pd.merge(ais_bikes, ais_lookup, how="left", on="AISPREDOT")

# Merging with AIS05TO98_CROSSWALK
ais_cross = puf_2022["PUF_AIS15TO05_CROSSWALK"]
ais_cross.columns = ['AIS15_PREDOT', 'AIS15_SEVERITY', 'AISPREDOT', 'AIS05_SEVERITY', 'AIS15_DESCRIPTION']
ais_cross["AISPREDOT"] = ais_cross["AISPREDOT"].replace("None", np.nan)
ais_cross["AISPREDOT"] = ais_cross["AISPREDOT"].astype(float)
ais_cross = ais_cross.drop("AIS05_SEVERITY", axis=1)
ais_bikes = pd.merge(ais_bikes, ais_cross, how="left", on="AISPREDOT")
ais_bikes["Year"] = 2022
stat_printer(ais_bikes)

# Optional export to CSV
#ais_bikes.to_csv(cwd+"/TQP_Files/ais/2022_AIS.csv", index=False)

### Hospital events

In [ ]:
# Filtering PUF_HOSPITALEVENTS for biker INC_KEYS
events = puf_2022["PUF_HOSPITALEVENTS"]
events_bikes = events[events["INC_KEY"].isin(inc_keys)]
events_bikes["Year"] = 2022
stat_printer(events_bikes)

# Optional export to CSV
#events_bikes.to_csv(cwd+"/TQP_Files/events/2022_events.csv", index=False)

### Diagnosis

In [ ]:
# Filtering PUF_AISDIAGNOSIS for biker INC_KEYS
diag = puf_2022["PUF_ICDDIAGNOSIS"]
diag_bikes = diag[diag["INC_KEY"].isin(inc_keys)]

# Merging with ICDDIAGNOSISCODE descriptions
diag_loopup = puf_2022["PUF_ICDDIAGNOSIS_LOOKUP"]
diag_loopup = diag_loopup.drop("ICD_VERSION", axis=1)
diag_bikes = pd.merge(diag_bikes, diag_loopup, how="left", on="ICDDIAGNOSISCODE")
diag_bikes["Year"] = 2022
stat_printer(diag_bikes)

# Optional export to CSV
#diag_bikes.to_csv(cwd+"/TQP_Files/diagnosis/2022_diagnosis.csv", index=False)

### Procedures

In [ ]:
# Filtering PUF_ICDPROCEDURE for biker INC_KEYS
proced = puf_2022["PUF_ICDPROCEDURE"]
proced_bikes = proced[proced["INC_KEY"].isin(inc_keys)]

# Merging with ICDPROCEDURECODE descriptions
proced_loopup = puf_2022["PUF_ICDPROCEDURE_LOOKUP"]
proced_loopup = proced_loopup.drop("ICD_VERSION", axis=1)
proced_bikes = pd.merge(proced_bikes, proced_loopup, how="left", on="ICDPROCEDURECODE")
proced_bikes["Year"] = 2022
stat_printer(proced_bikes)

# Optional export to CSV
#proced_bikes.to_csv(cwd+"/TQP_Files/procedures/2022_procedures.csv", index=False)

### Preexisting Conditions

In [ ]:
# Filtering PUF_HOSPITALEVENTS for biker INC_KEYS
pre = puf_2022["PUF_PREEXISTINGCONDITIONS"]
pre_bikes = pre[pre["INC_KEY"].isin(inc_keys)]
pre_bikes["Year"] = 2022
stat_printer(pre_bikes)

# Optional export to CSV
#pre_bikes.to_csv(cwd+"/TQP_Files/preexisting/2022_preexisting_conditions.csv", index=False)